In [1]:
from tqdm import tqdm
import cv2
import numpy as np

from utils import list_images
from yunet import YuNet
from sface import SFace

You can download the LFW dataset here:
http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
http://vis-www.cs.umass.edu/lfw/#download

in this dataset we are going to have some duplicate samples, but even with these duplicate data you can clearly see tht the threshold is high enought.

In [2]:
IMAGES_PATH = "/home/rodrigo/Downloads/lfw-deepfunneled"
USE_YUNET = False
THRESHOLD = 0.363  # If we use something above 0.55, the rate of false positive will be below 2%

In [3]:
weights_path = "./sface/bins/face_recognition_sface_2021dec.onnx"
recognizer = SFace(model_path=weights_path, dis_type=0, backend_id=cv2.dnn.DNN_BACKEND_CUDA,
                   target_id=cv2.dnn.DNN_TARGET_CUDA)

face_detector = YuNet()

In [4]:
images_path = list_images(IMAGES_PATH)

hashs = []
for fname in tqdm(list(images_path)):
    image = cv2.imread(fname)
    if USE_YUNET:
        try:  # not all images are going to find some face
            data = face_detector.detect(image)[0][1]
            face_hash = recognizer.infer(image, data)
        except:
            continue
    else:
        face_hash = recognizer.infer(image)
    hashs.append({"hash": cv2.normalize(face_hash, None), "fname": fname})

100%|██████████| 5748/5748 [00:38<00:00, 149.38it/s]


In [5]:
dist_mat = np.zeros((len(hashs), len(hashs)))
for i in tqdm(range(len(hashs))):
    for j in range(len(hashs)):
        if i == j:  # we don't want the same image to be compared with itself
            continue
        dist_mat[i, j] = (hashs[i]["hash"] * hashs[j]["hash"]).sum()  # cosine distance

100%|██████████| 5748/5748 [01:23<00:00, 68.91it/s]


In [6]:
# get best match
x, y = np.unravel_index(dist_mat.argmax(), dist_mat.shape)

print(hashs[x]["fname"])
print(hashs[y]["fname"])
print(x, y, dist_mat.max())

cv2.imshow("image", cv2.imread(hashs[x]["fname"]))
cv2.imshow("image2", cv2.imread(hashs[y]["fname"]))
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/rodrigo/Downloads/lfw-deepfunneled/Raul_Ibanez/Raul_Ibanez_0001.jpg
/home/rodrigo/Downloads/lfw-deepfunneled/Carlos_Beltran/Carlos_Beltran_0001.jpg
618 1785 0.9866505861282349


Qt: Session management error: Could not open network socket


In [7]:
# does the same match with the opencv detector, just to compare if the results are the same
recognizer._model.match(hashs[x]["hash"], hashs[y]["hash"])

0.9866505706308999

In [8]:
# how many false positives are there?
rank = {}
matches = {}
for i in range(dist_mat.shape[0]):
    for j in range(dist_mat.shape[1]):
        if dist_mat[i, j] > THRESHOLD:
            if i not in rank:
                rank[i] = 1
                matches[i] = [(j, dist_mat[i, j])]
            else:
                rank[i] += 1
                matches[i].append((j, dist_mat[i, j]))

keys = list(rank.keys())

print("False positives:", len(keys), len(keys) / len(hashs))  # We suppose that there are only picture for each person

False positives: 5748 1.0


In [9]:
# Which sample has more false positives?
sorted_rank = dict(sorted(rank.items(), key=lambda x: x[1], reverse=True))
sorted_rank

{2926: 4956,
 4241: 4935,
 4150: 4928,
 3161: 4898,
 4710: 4888,
 4587: 4887,
 2145: 4874,
 2134: 4864,
 5344: 4851,
 412: 4847,
 184: 4805,
 1084: 4794,
 2086: 4778,
 3753: 4758,
 3783: 4752,
 1393: 4750,
 4614: 4742,
 636: 4728,
 5099: 4716,
 1669: 4710,
 713: 4699,
 1816: 4695,
 4916: 4674,
 954: 4668,
 3066: 4663,
 3774: 4641,
 4641: 4639,
 2685: 4636,
 1953: 4634,
 5527: 4632,
 4774: 4621,
 1212: 4617,
 4069: 4617,
 585: 4615,
 1866: 4609,
 4900: 4604,
 2419: 4597,
 3903: 4595,
 3923: 4593,
 5053: 4590,
 3035: 4587,
 3287: 4585,
 4829: 4584,
 1813: 4583,
 2666: 4578,
 1759: 4575,
 1972: 4574,
 1656: 4570,
 2066: 4564,
 1014: 4559,
 2095: 4556,
 1852: 4554,
 2910: 4554,
 4007: 4549,
 1476: 4546,
 1134: 4545,
 3195: 4545,
 1367: 4542,
 2270: 4538,
 1762: 4533,
 1767: 4532,
 2607: 4530,
 2551: 4529,
 2627: 4521,
 6: 4519,
 5385: 4519,
 5500: 4518,
 4204: 4517,
 4671: 4517,
 2570: 4512,
 3051: 4511,
 3976: 4501,
 2826: 4500,
 916: 4499,
 1221: 4495,
 5206: 4493,
 5283: 4493,
 5450: 44

In [15]:
# show it
selected = list(sorted_rank.keys())[0]
worst = matches[selected]

ref_image = cv2.imread(hashs[selected]["fname"])
cv2.imshow("ref", ref_image)

h, w, c = ref_image.shape
mosaic = np.zeros((h * 5, w * 5, c), dtype=ref_image.dtype)
for i, sample in enumerate(worst[:25]):
    idx = sample[0]
    dist = sample[1]
    im = cv2.imread(hashs[idx]["fname"])
    cv2.putText(im, f"{dist:.3f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    y = i // 5
    x = i % 5
    mosaic[y * h:y * h + h, x * w:x * w + w, :] = im
    
cv2.imshow("face", cv2.resize(mosaic, None, fx=0.5, fy=0.5))
cv2.waitKey(0)
#cv2.destroyAllWindows()

225